Santhoshi Veera Haritha Vytla(vytlasa)
Djeinaba Ba(badb)
Mounica Ragam(ragamma)

In [7]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
import numpy as np

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

def sigmoid(activation):
	return 1.0 / (1.0 + np.exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = sigmoid(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def sigmoid_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

def load_and_prepare_data():
    # Test Backprop on Seeds dataset
    seed(1)
    # load and prepare data
    filename = 'data_banknote_authentication.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])-1):
        str_column_to_float(dataset, i)
    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    # normalize input variables
    minmax = dataset_minmax(dataset)
    #normalize_dataset(dataset, minmax)
    return dataset,minmax

dataset,minmax=load_and_prepare_data()
# evaluate algorithm
def optimal_layers():
    n_folds = 5
    l_rate = 0.001
    n_epoch = 100
    n_hidden = 5
    final_scores=list()
    i=1
    while(i<=n_hidden):
        scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, i)
        final_scores.append(sum(scores)/float(len(scores)))
        i+=1
    return final_scores
final_score = optimal_layers()
print("--------------------------------------------------------------------------")
print("Sigmoid for forward propagation, Sigmoid derivative for back propagation")
print("--------------------------------------------------------------------------")
print("Accuracies")
print("--------------------------------------------------------------")
for i in range (0,5):
    print("Hidden Layer - " + str(i+1) + ":" + str(final_score[i]))
print("---------------------------------------------------------------")
print("Max Accuracy - "+ str(max(final_score)))
print("------------------------------------------------------------")
print("Best Size of the hidden layer - "+str(final_score.index(max(final_score))+1))

--------------------------------------------------------------------------
Sigmoid for forward propagation, Sigmoid derivative for back propagation
--------------------------------------------------------------------------
Accuracies
--------------------------------------------------------------
Hidden Layer - 1:96.56934306569343
Hidden Layer - 2:96.2043795620438
Hidden Layer - 3:96.42335766423358
Hidden Layer - 4:96.27737226277372
Hidden Layer - 5:95.91240875912408
---------------------------------------------------------------
Max Accuracy - 96.56934306569343
------------------------------------------------------------
Best Size of the hidden layer - 1


In [8]:
# Transfer neuron activation
def tanh(activation):
	return (np.exp(activation) - np.exp(-activation))/(np.exp(activation) + np.exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = tanh(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def tanh_derivative(output):
	return 1.0 - (output*output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * tanh_derivative(neuron['output'])

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

dataset,minmax=load_and_prepare_data()
# evaluate algorithm

def optimal_layers():
    n_folds = 5
    l_rate = 0.001
    n_epoch = 100
    n_hidden = 5
    final_scores=list()
    i=1
    while(i<=n_hidden):
        scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, i)
        final_scores.append(sum(scores)/float(len(scores)))
        i+=1
    return final_scores
print("--------------------------------------------------------------------------")
print("Tanh for forward propagation, Tanh derivative for back propagation")
print("--------------------------------------------------------------------------")
print("Accuracies")
print("--------------------------------------------------------------")
for i in range (0,5):
    print("Hidden Layer - " + str(i) + ":" + str(final_score[i]))
print("---------------------------------------------------------------")
print("Max Accuracy - "+ str(max(final_score)))
print("------------------------------------------------------------")
print("Best Size of the hidden layer - "+str(final_score.index(max(final_score))+1))

--------------------------------------------------------------------------
Tanh for forward propagation, Tanh derivative for back propagation
--------------------------------------------------------------------------
Accuracies
--------------------------------------------------------------
Hidden Layer - 0:96.56934306569343
Hidden Layer - 1:96.2043795620438
Hidden Layer - 2:96.42335766423358
Hidden Layer - 3:96.27737226277372
Hidden Layer - 4:95.91240875912408
---------------------------------------------------------------
Max Accuracy - 96.56934306569343
------------------------------------------------------------
Best Size of the hidden layer - 1


In [9]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = sigmoid(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * tanh_derivative(neuron['output'])

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

dataset,minmax=load_and_prepare_data()
# evaluate algorithm
def optimal_layers():
    n_folds = 5
    l_rate = 0.001
    n_epoch = 100
    n_hidden = 5
    final_scores=list()
    i=1
    while(i<=n_hidden):
        scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, i)
        final_scores.append(sum(scores)/float(len(scores)))
        i+=1
    return final_scores
final_score = optimal_layers()
print("--------------------------------------------------------------------------")
print("Sigmoid for forward propagation, Tanh derivative for back propagation")
print("--------------------------------------------------------------------------")
print("Accuracies")
print("--------------------------------------------------------------")
for i in range (0,5):
    print("Hidden Layer - " + str(i) + ":" + str(final_score[i]))
print("---------------------------------------------------------------")
print("Max Accuracy - "+ str(max(final_score)))
print("------------------------------------------------------------")
print("Best Size of the hidden layer - "+str(final_score.index(max(final_score))+1))

--------------------------------------------------------------------------
Sigmoid for forward propagation, Tanh derivative for back propagation
--------------------------------------------------------------------------
Accuracies
--------------------------------------------------------------
Hidden Layer - 0:84.5985401459854
Hidden Layer - 1:89.78102189781023
Hidden Layer - 2:89.78102189781022
Hidden Layer - 3:89.4890510948905
Hidden Layer - 4:92.11678832116789
---------------------------------------------------------------
Max Accuracy - 92.11678832116789
------------------------------------------------------------
Best Size of the hidden layer - 5


In [10]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = tanh(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output'])

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

dataset,minmax=load_and_prepare_data()
# evaluate algorithm
def optimal_layers():
    n_folds = 5
    l_rate = 0.001
    n_epoch = 100
    n_hidden = 5
    final_scores=list()
    i=1
    while(i<=n_hidden):
        scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, i)
        final_scores.append(sum(scores)/float(len(scores)))
        i+=1
    return final_scores
final_score = optimal_layers()
print("--------------------------------------------------------------------------")
print("Tanh for forward propagation, Sigmoid derivative for back propagation")
print("--------------------------------------------------------------------------")
for i in range (0,5):
    print("Hidden Layer - " + str(i) + ":" + str(final_score[i]))
print("---------------------------------------------------------------")
print("Max Accuracy - "+ str(max(final_score)))
print("------------------------------------------------------------")
print("Best Size of the hidden layer - "+str(final_score.index(max(final_score))+1))

C:\Users\bnama\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


--------------------------------------------------------------------------
Tanh for forward propagation, Sigmoid derivative for back propagation
--------------------------------------------------------------------------
Hidden Layer - 0:49.416058394160586
Hidden Layer - 1:55.54744525547445
Hidden Layer - 2:55.54744525547445
Hidden Layer - 3:55.54744525547445
Hidden Layer - 4:55.47445255474453
---------------------------------------------------------------
Max Accuracy - 55.54744525547445
------------------------------------------------------------
Best Size of the hidden layer - 2
